In [7]:
room_config = {
    "officeroom": {
        "north": "wall",
        "east": "livingroom",
        "south": "wall",
        "west": "wall",
    },
    "livingroom": {
        "north": "wall",
        "east": "wall",
        "south": "bedroom",
        "west": "officeroom",
    },
    "bedroom": {
        "north": "livingroom",
        "east": "wall",
        "south": "wall",
        "west": "wall",
    },
}


object_transition_config = {
    "static": {"bed": None, "desk": None, "table": None},
    "independent": {
        "tae": {
            "officeroom": {"north": 0, "east": 0.1, "south": 0, "west": 0, "stay": 0.9},
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0.1,
                "stay": 0.9,
            },
            "bedroom": {"north": 0, "east": 0, "south": 0, "west": 0, "stay": 0},
        },
        "michael": {
            "officeroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.9,
                "west": 0,
                "stay": 0.1,
            },
            "bedroom": {"north": 0.1, "east": 0, "south": 0, "west": 0, "stay": 0.9},
        },
        "vincent": {
            "officeroom": {
                "north": 0,
                "east": 0.5,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.333,
                "west": 0.333,
                "stay": 0.333,
            },
            "bedroom": {
                "north": 0.5,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
        },
    },
    "dependent": {
        "laptop": {"tae": 0.7, "michael": 0.4, "vincent": 0.1},
        "phone": {"tae": 0.1, "michael": 0.7, "vincent": 0.4},
        "headset": {"tae": 0.4, "michael": 0.1, "vincent": 0.9},
    },
    "agent": {
        "agent": {"officeroom": None, "livingroom": None, "bedroom": None},
    },
}

object_init_config = {
    "static": {
        "bed": {"officeroom": 0, "livingroom": 0, "bedroom": 1},
        "desk": {"officeroom": 1, "livingroom": 0, "bedroom": 0},
        "table": {"officeroom": 0, "livingroom": 1, "bedroom": 0},
    },
    "independent": {
        "tae": {"officeroom": 0.5, "livingroom": 0.5, "bedroom": 0},
        "michael": {"officeroom": 0, "livingroom": 0.5, "bedroom": 0.5},
        "vincent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "dependent": {
        "laptop": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "phone": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "headset": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "agent": {
        "agent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
}

config = {
    "room_config": room_config,
    "object_transition_config": object_transition_config,
    "object_init_config": object_init_config,
    "question_prob": 1.0,
    "seed": 42,
    "terminates_at": 99,
}


In [10]:
import gymnasium as gym
import random

env = gym.make("room_env:RoomEnv-v2", **config)
(obs, question), info = env.reset()
rewards = []
obs_all = []
while True:
    action_qa = "livingroom"
    action_explore = random.choice(["north", "east", "south", "west", "stay"])
    (obs, question), reward, done, truncated, info = env.step(("wall", action_explore))
    rewards.append(reward)
    obs_all.append(obs)
    if done:
        break

/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:491: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:156: UserWarning: WARN: The obs returned by the `res

In [13]:
max([len(obs) for obs in obs_all]), min([len(obs) for obs in obs_all])


(11, 6)

In [14]:
obs, question

([['livingroom', 'tothenorth', 'wall', 100],
  ['livingroom', 'totheeast', 'wall', 100],
  ['livingroom', 'tothesouth', 'bedroom', 100],
  ['livingroom', 'tothewest', 'officeroom', 100],
  ['table', 'atlocation', 'livingroom', 100],
  ['vincent', 'atlocation', 'livingroom', 100],
  ['headset', 'atlocation', 'livingroom', 100],
  ['agent', 'atlocation', 'livingroom', 100]],
 ['table', 'atlocation', '?', 100])

In [15]:
obs


[['livingroom', 'tothenorth', 'wall', 100],
 ['livingroom', 'totheeast', 'wall', 100],
 ['livingroom', 'tothesouth', 'bedroom', 100],
 ['livingroom', 'tothewest', 'officeroom', 100],
 ['table', 'atlocation', 'livingroom', 100],
 ['vincent', 'atlocation', 'livingroom', 100],
 ['headset', 'atlocation', 'livingroom', 100],
 ['agent', 'atlocation', 'livingroom', 100]]

In [4]:
env.hidden_global_state

[['officeroom', 'tothenorth', 'wall', 100],
 ['officeroom', 'totheeast', 'livingroom', 100],
 ['officeroom', 'tothesouth', 'wall', 100],
 ['officeroom', 'tothewest', 'wall', 100],
 ['livingroom', 'tothenorth', 'wall', 100],
 ['livingroom', 'totheeast', 'wall', 100],
 ['livingroom', 'tothesouth', 'bedroom', 100],
 ['livingroom', 'tothewest', 'officeroom', 100],
 ['bedroom', 'tothenorth', 'livingroom', 100],
 ['bedroom', 'totheeast', 'wall', 100],
 ['bedroom', 'tothesouth', 'wall', 100],
 ['bedroom', 'tothewest', 'wall', 100],
 ['bed', 'atlocation', 'bedroom', 100],
 ['desk', 'atlocation', 'officeroom', 100],
 ['table', 'atlocation', 'livingroom', 100],
 ['tae', 'atlocation', 'officeroom', 100],
 ['michael', 'atlocation', 'bedroom', 100],
 ['vincent', 'atlocation', 'livingroom', 100],
 ['laptop', 'atlocation', 'officeroom', 100],
 ['phone', 'atlocation', 'bedroom', 100],
 ['headset', 'atlocation', 'livingroom', 100],
 ['agent', 'atlocation', 'livingroom', 100]]

In [17]:
import datetime
import os
import random
from copy import deepcopy
from typing import Dict, List, Tuple

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from tqdm.auto import tqdm, trange

from explicit_memory.memory import EpisodicMemory, SemanticMemory, ShortMemory
from explicit_memory.policy import answer_question, encode_observation, manage_memory
from explicit_memory.utils import ReplayBuffer, is_running_notebook, write_yaml


class HandCraftedAgent:
    """Handcrafted agent interacting with environment.

    This agent explores the roooms, i.e., KGs. The exploration can be uniform-random,
    or just avoiding walls.

    """

    def __init__(
        self,
        env_str: str = "room_env:RoomEnv-v2",
        env_config: dict = None,
        memory_management_policy: str = "episodic_semantic",
        qa_policy: str = "episodic_semantic",
        explore_policy: str = "random",
        num_samples_for_results: int = 10,
        capacity: dict = {
            "episodic": 4,
            "semantic": 4,
            "short": 16,
        },
    ) -> None:
        """Initialize the agent.

        Args
        ----
        env_str: This has to be "room_env:RoomEnv-v2"
        env_config: The configuration of the environment.
        memory_management_policy: Memory management policy.
        qa_policy: question answering policy
        policy: The room exploration policy. Choose one of "random", "avoid_walls"
        num_samples_for_results: The number of samples to validate / test the agent.
        capacity: The capacity of each human-like memory systems.

        """
        self.all_params = deepcopy(locals())
        del self.all_params["self"]
        self.env_str = env_str
        self.env_config = env_config
        self.memory_management_policy = memory_management_policy
        self.qa_policy = qa_policy
        self.explore_policy = explore_policy
        self.num_samples_for_results = num_samples_for_results
        self.capacity = capacity

        self.env = gym.make(self.env_str, **env_config)

        self.default_root_dir = f"./training_results/{str(datetime.datetime.now())}"
        os.makedirs(self.default_root_dir, exist_ok=True)

    def init_memory_systems(self, num_actions: int = 5) -> None:
        """Initialize the agent's memory systems. This has nothing to do with the
        replay buffer."""
        self.action_space = gym.spaces.Discrete(num_actions)
        self.memory_systems = {
            "episodic": EpisodicMemory(capacity=self.capacity["episodic"]),
            "semantic": SemanticMemory(capacity=self.capacity["semantic"]),
            "short": ShortMemory(capacity=self.capacity["short"]),
        }

    def get_memory_state(self) -> dict:
        """Return the current state of the memory systems. This is NOT what the gym env
        gives you. This is made by the agent.

        """
        state_as_dict = {
            "episodic": self.memory_systems["episodic"].return_as_lists(),
            "semantic": self.memory_systems["semantic"].return_as_lists(),
            "short": self.memory_systems["short"].return_as_lists(),
        }
        return state_as_dict

    def encode_all_observations(self, observations: List[List[str]]) -> None:
        """Encode all observations to the short-term memory systems.

        Args
        ----
        observations: A list of list of quadruples.

        """
        for obs in observations:
            encode_observation(self.memory_systems, obs)

    def test(self):
        """Test the agent. There is no training for this agent, since it is
        handcrafted."""
        self.scores = []
        for _ in range(self.num_samples_for_results):
            self.init_memory_systems()
            (observations, self.question), info = self.env.reset()
            self.encode_all_observations(observations)

            self.manage_memory(observations)

            done = False
            score = 0
            while not done:
                if self.policy.lower() == "random":
                    selected_action = random.choice(["episodic", "semantic", "forget"])
                    manage_memory(self.memory_systems, selected_action)
                    qa_policy = "episodic_semantic"
                elif self.policy.lower() == "episodic_only":
                    manage_memory(self.memory_systems, "episodic")
                    qa_policy = "episodic"
                elif self.policy.lower() == "semantic_only":
                    qa_policy = "semantic"
                    manage_memory(self.memory_systems, "semantic")
                else:
                    raise ValueError("Unknown policy.")

                answer = str(
                    answer_question(self.memory_systems, qa_policy, self.question)
                ).lower()
                (
                    (observation, self.question),
                    reward,
                    done,
                    truncated,
                    info,
                ) = self.env.step(answer)

                encode_observation(self.memory_systems, observation)
                score += reward
            self.scores.append(score)

        results = {
            "test_score": {
                "mean": round(np.mean(self.scores).item(), 2),
                "std": round(np.std(self.scores).item(), 2),
            }
        }
        write_yaml(results, os.path.join(self.default_root_dir, "results.yaml"))
        write_yaml(self.all_params, os.path.join(self.default_root_dir, "train.yaml"))
        write_yaml(
            self.get_memory_state(),
            os.path.join(self.default_root_dir, "last_memory_state.yaml"),
        )


ModuleNotFoundError: No module named 'nn'

In [25]:
def foo(foo: List[List[str]]) -> None:
    print(foo)


foo([["asdf", "bar", "baz", "qux"]])

[['asdf', 'bar', 'baz', 'qux']]
